In [1]:
import openai 
import requests 
import numpy as np 
from retrying import retry
from typing import List, Optional, Union

install and import basic package for GPT

In [2]:
!python3 -m pip install pandas bs4 wikipedia sqlalchemy torch matplotlib
import pandas as pd 
import requests
from bs4 import BeautifulSoup
import wikipedia
import os,sys,pathlib
import cv2
import datetime
import sqlalchemy
import torch
import matplotlib.pyplot as plt


[notice] A new release of pip is available: 23.0.1 -> 23.2
[notice] To update, run: python3.11 -m pip install --upgrade pip


## OpenAI API Setting

In [3]:
import openai
import logging
from termcolor import colored
# load from key file
with open('./rilab_key.txt') as f:
    OPENAI_API_KEY = key = f.read()
openai.api_key = OPENAI_API_KEY

## GPTChat Class 


In [4]:
class GPTChatClass:
    def __init__(self, model="gpt-4", role_msg = 'You are a helpful assistant.'):
        self.model = model
        self.messages = [{"role": "system", "content": f'{role_msg}'}]  # initial system message
        self.response = None
        self.logger = logging.getLogger(__name__)

    def add_message(self, role, content):
        if not role or not content:
            self.logger.error("Role or content is missing.")
            return
        self.messages.append({"role": role, "content": content})

    @retry(stop_max_attempt_number=7, wait_exponential_multiplier=1000, wait_exponential_max=10000)
    def __call__(self, user_input : str = 'hi'):
        if not user_input:
            self.logger.error("User input is missing.")
            return
        self.add_message("user", user_input)
        try:
            self.response = openai.ChatCompletion.create(
                model=self.model,
                messages=self.messages
            )
        except Exception as e:
            self.logger.error(f"Failed to query the model: {e}")
            return None
        self.add_message("assistant", self.get_response_content())  # save the assistant's response
        return self.get_response_content()

    def get_response_content(self):
        if self.response:
            return self.response["choices"][0]["message"]["content"]
        else:
            self.logger.warning("Response is empty.")
            return None

    def get_response_status(self):
        if self.response:
            return self.response["choices"][0]["finish_reason"]
        else:
            self.logger.warning("Response is empty.")
            return None
    
    def print_history(self):
        for message in self.messages:
            role = message["role"]
            content = message["content"]

            if role == "user":
                formatted_message = colored("User : ", attrs=["bold"]) + content
            elif role == "assistant":
                formatted_message = colored("Assistant : ", attrs=["bold"]) + colored(content, "green", "on_dark_grey")
            else:
                #formatted_message = content  # System message, no formatting
                continue

            print(formatted_message)
            print("="*50)

In [5]:
gptchat = GPTChatClass(role_msg = 'You are a helpful assistant.')
gptchat(user_input = 'hi')
# Assitant : Hello! How can I assist you today?
gptchat(user_input = 'Who is the current korean president?')
# Assitant : The current president of South Korea, as of 2022, is Moon Jae-in. He has been in office since May 10, 2017.

'The current president of South Korea, as of my latest update, is Moon Jae-in. He assumed office on May 10, 2017. However, I recommend checking the latest sources to make sure the information is still current.'

### Print Chat History

In [6]:
gptchat.print_history()

User : hi
Assistant : Hello! How can I assist you today?
User : Who is the current korean president?
Assistant : The current president of South Korea, as of my latest update, is Moon Jae-in. He assumed office on May 10, 2017. However, I recommend checking the latest sources to make sure the information is still current.


## Make GPT use Jupyter Notebook


In [82]:
import nbformat
from nbconvert.preprocessors import ExecutePreprocessor
from nbconvert.preprocessors.execute import CellExecutionError

# Create a new notebook
nb = nbformat.v4.new_notebook()

# Add a cell with your code
code_str = """
import sys

print('hi)
"""
code_cell = nbformat.v4.new_code_cell(source=code_str)
nb.cells.append(code_cell)

# Execute the notebook
ep = ExecutePreprocessor(timeout=600, kernel_name='python3')
output_str, error_str =None,None
try:
    ep.preprocess(nb)
    output = nb.cells[0].outputs[0]
    #print(output)
    output_str = output['text']
except CellExecutionError as e:
    error_str = e


0.00s - Debugger warning: It seems that frozen modules are being used, which may
0.00s - make the debugger miss breakpoints. Please pass -Xfrozen_modules=off
0.00s - to python to disable frozen modules.
0.00s - Note: Debugging will proceed. Set PYDEVD_DISABLE_FILE_VALIDATION=1 to disable this validation.


*Error Message* cleanning

In [83]:
if error_str is not None:
    # Get the traceback, which is a list of strings, and join them into one string
    filtered_error_msg = error_str.__str__().split('An error occurred while executing the following cell')[-1].split("\n------------------\n")[-1]
    raw_error_msg = "".join(filtered_error_msg)
    
    # Remove escape sequences for colored text
    error_msg = raw_error_msg.replace("\x1b[0m", "").replace("\x1b[0;31m", "").replace("\x1b[0;32m", "").replace("\x1b[1;32m", "").replace("\x1b[38;5;241m", "").replace("\x1b[38;5;28;01m", "").replace("\x1b[38;5;21m", "").replace("\x1b[38;5;28m", "").replace("\x1b[43m", "").replace("\x1b[49m", "").replace("\x1b[38;5;241;43m", "").replace("\x1b[39;49m", "").replace("\x1b[0;36m", "")
    error_lines = error_msg.split("\n")
    
    # Only keep the lines up to (and including) the first line that contains 'Error' followed by a ':'
    error_lines = error_lines[:next(i for i, line in enumerate(error_lines) if 'Error:' in line) + 1]

    # Join the lines back into a single string
    error_msg = "\n".join(error_lines)
    
    print('-----[ERROR]-----')
    print(error_msg)
else:
    print('-----[OUTPUT]-----')
    print(output_str)


-----[ERROR]-----

  Cell In[1], line 3
    print('hi)
          ^
SyntaxError: unterminated string literal (detected at line 3)
